In [1]:
from SuperLore.lorem_new import LOREM
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from collections import defaultdict

In [2]:
train_set = pd.read_csv('train_set.csv')
train_label = pd.read_csv('train_label.csv')
test_set = pd.read_csv('test_set.csv')
test_label = pd.read_csv('test_label.csv')
train_set.pop('Unnamed: 0')
train_label.pop('Unnamed: 0')
test_set.pop('Unnamed: 0')
test_label.pop('Unnamed: 0')

0       10583
1       16809
2       11025
3       19306
4        4521
        ...  
2692    18374
2693    19220
2694    14871
2695      858
2696     1648
Name: Unnamed: 0, Length: 2697, dtype: int64

In [3]:
rf = RandomForestClassifier(bootstrap=False, class_weight='balanced',
                       criterion='gini', max_depth=500, max_features='auto',
                       max_leaf_nodes=None, min_impurity_decrease=0.0, min_samples_leaf=10,
                       min_samples_split=50, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=None, oob_score=False,
                       random_state=0, verbose=0, warm_start=False)
rf.fit(train_set, train_label)



RandomForestClassifier(bootstrap=False, class_weight='balanced', max_depth=500,
                       max_features='auto', min_samples_leaf=10,
                       min_samples_split=50, random_state=0)

In [4]:
predictions1 = rf.predict(train_set)
score = rf.score(train_set, train_label)
report = classification_report(train_label, predictions1)
print('Classification report for train set ', report)
predictions1 = rf.predict(test_set)
score = rf.score(test_set, test_label)
report = classification_report(test_label, predictions1)
print('Classification report for test set ', report)


Classification report for train set                precision    recall  f1-score   support

           0       0.96      0.81      0.88      4730
           1       0.61      0.91      0.73      1561

    accuracy                           0.83      6291
   macro avg       0.79      0.86      0.80      6291
weighted avg       0.88      0.83      0.84      6291

Classification report for test set                precision    recall  f1-score   support

           0       0.95      0.79      0.86      2028
           1       0.58      0.87      0.69       669

    accuracy                           0.81      2697
   macro avg       0.76      0.83      0.78      2697
weighted avg       0.86      0.81      0.82      2697



In [5]:
neigh_type = 'cfs' # the generation you want (random, genetic, geneticp, cfs, rndgen)
binary = 'binary_from_dts' #how to merge the trees (binary from dts, binary from bb are creating a binary tree, nari is creating a n ari tree)
cxpb = 0.7 # values to set for the genetic generation
mutpb = 0.5 #values to set for the genetic generation
ngen = 2 # number of neighborhood generations to do
runs = 2 #how many neighbours and trees to create and then merge
class_name = 'class' #name of the column used as target
class_values = [0,1] #values that the target may have

feature_names = test_set.columns
real_feature_names = test_set.columns
numeric_columns = list()
for c in test_set.columns:
    numeric_columns.append(test_set.columns.get_loc(c))

features_map = defaultdict(dict)
i = 0
j = 0

while i < len(feature_names) and j < len(real_feature_names):
    if feature_names[i] == real_feature_names[j]:
        features_map[j][feature_names[i].replace('%s=' % real_feature_names[j], '')] = i
        i += 1
        j += 1
    elif feature_names[i].startswith(real_feature_names[j]):
        features_map[j][feature_names[i].replace('%s=' % real_feature_names[j], '')] = i
        i += 1
    else:
        j += 1

In [6]:
neigh_kwargs = {
        "balance": False,
        "sampling_kind": "gaussian",
        "kind": "gaussian_global",
        "downward_only": True,
        "redo_search": True,
        "forced_balance_ratio": 0.5,
        "cut_radius": True,
        "n": 800,
        "normalize": 'minmax',
        "forced_balance_ratio": 0.5,
        "n_batch": 5,
        "datas": train_set.values
    }

In [7]:
# neigh type: the way in which you want to generate the neighbourhood (random, genetic, geneticp, rndgen, rndgenp, cfs)
# ocr, mutpb, cxpb: values to set for the genetic generation
# negen: number of neighborhood generations to do
# runs: how many neighbours and trees to create and then merge
# discretize: in the surrogate trees, discretize variables to better generalize and have a smaller tree
# binary: how to merge the trees (binary from dts, binary from bb are creating a binary tree, nari is creating a n ari tree)
# encdec: if you want to generate the neighbours in an encoded space (for now you can select onw hot and target encoding)
# dataset: in case you selected an encdec, this dataset is used to fit the encoder
# K_transformed: a piece of dataset in the form you can feed to the BB
# extreme fidelity: if True, it checks that the prediction of the surrogate model is the same as the BB
# filter c rules: it filters the counterfactual rules by checking if they are true conterfactuals or not
explainer = LOREM(train_set.values, rf.predict, rf.predict_proba, feature_names, class_name, class_values, numeric_columns, features_map,
                      neigh_type=neigh_type, categorical_use_prob=True, continuous_fun_estimation=True, size=1000,
                      ocr=0.1, multi_label=False, one_vs_rest=False, random_state=42, verbose=True,
                      Kc=train_set, bb_predict_proba=rf.predict_proba, K_transformed=train_set, discretize=True,
                      encdec=None, binary=binary, **neigh_kwargs)

Neigh kind  cfs
sampling kind  gaussian_global


In [8]:
# x the instance to explain
# samples the number of samples to generate during the neighbourhood generation
# use weights True or False
# metric default is neuclidean, it is the metric employed to measure the distance between records
# runs number of times the neighbourhood generation is done
# exemplar_num number of examplars to retrieve
# kwargs a dictionary in which add the parameters needed for cfs generation
# kernel and kernel width are for the definition of weights. deafult is None, it automatically select them.

explanation = explainer.explain_instance_stable(test_set.loc[0].values.reshape(1,-1), 150, runs=runs, n_jobs=2)

binary sampling search  800 5
binary sampling search  800 5


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   21.4s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:   21.4s finished


Synthetic neighborhood class counts {0: 281, 1: 281}
Synthetic neighborhood class counts {0: 285, 1: 285}
Learning local decision trees


[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=2)]: Done   2 out of   2 | elapsed:    0.0s finished


entro con exemplars  [[    26 167350     10 ...      0      0      0]
 [    50 166419      7 ...      0      0      0]
 [    40 168071     11 ...      0      0      0]
 ...
 [    63 247986     15 ...      0      0      0]
 [    35 248010     13 ...      0      0      0]
 [    36 248010      9 ...      0      0      0]] ['age', ' fnlwgt', ' education-num', ' capital-gain', ' capital-loss', ' hours-per-week', '0_1', '0_2', '0_3', '0_4', '0_5', '0_6', '0_7', '0_8', '0_9', '1_1', '1_2', '1_3', '1_4', '1_5', '1_6', '1_7', '1_8', '1_9', '1_10', '1_11', '1_12', '1_13', '1_14', '1_15', '1_16', '2_1', '2_2', '2_3', '2_4', '2_5', '2_6', '2_7', '3_1', '3_2', '3_3', '3_4', '3_5', '3_6', '3_7', '3_8', '3_9', '3_10', '3_11', '3_12', '3_13', '3_14', '3_15', '4_1', '4_2', '4_3', '4_4', '4_5', '4_6', '5_1', '5_2', '5_3', '5_4', '5_5', '6_1', '6_2', '7_1', '7_2', '7_3', '7_4', '7_5', '7_6', '7_7', '7_8', '7_9', '7_10', '7_11', '7_12', '7_13', '7_14', '7_15', '7_16', '7_17', '7_18', '7_19', '7_20', '7_21

In [9]:
print(explanation.rule)
for c in explanation.crules:
    print(c)

{  capital-gain <= 5392.20, age > -5234.82,  education-num > -1891.12, 2_2 <= 3858.73, 3_4 <= 7763.35, 3_2 > -1380.63, 4_4 > -2112.57, cluster <= 9905.75 } --> { class: 0 }
{  capital-gain > 5392.20, age > -5234.82, 2_2 <= 3858.73,  education-num > -1891.12, 0_3 > -163404.34, 4_2 <= 2130.07, 7_2 <= 163347.74, 7_27 > -217394.15, 4_6 <= 111135.48, 6_1 > -159186.62, 2_4 > -191364.49 } --> { class: 1 }
{  capital-gain <= 5392.20, age > -5234.82,  education-num > -1891.12, 2_2 <= 3858.73, 3_4 <= 7763.35, 3_2 > -1380.63, 4_4 <= -2112.57, 1_2 <= 66185.65 } --> { class: 1 }
{  capital-gain <= 5392.20, age > -5234.82,  education-num > -1891.12, 2_2 <= 3858.73, 3_4 > 7763.35, 7_37 > -92427.73, 1_1 <= 218112.29, 7_32 <= 176366.04 } --> { class: 1 }
{  capital-gain <= 5392.20, age > -5234.82,  education-num > -1891.12, 2_2 > 3858.73, 4_5 <= 75242.37, 4_1 > -187158.88, 3_3 > -19348.65, 7_16 > -166719.10,  capital-loss <= 4701.45, 3_4 <= 134867.96, 3_2 > -1380.63 } --> { class: 1 }


In [10]:
explanation.exemplars

'{ age = 26 fnlwgt = 167350 education-num = 10 capital-gain = 3103 hours-per-week = 400_3 = 11_6 = 12_2 = 13_11 = 14_2 = 15_1 = 16_1 = 17_1 = }\n{ age = 50 fnlwgt = 166419 education-num = 7 capital-gain = 3674 hours-per-week = 400_4 = 11_3 = 12_1 = 13_6 = 14_1 = 15_2 = 16_2 = 17_1 = }\n{ age = 40 fnlwgt = 168071 education-num = 11 capital-gain = 3325 hours-per-week = 400_3 = 11_8 = 12_3 = 13_11 = 14_1 = 15_1 = 16_1 = 17_1 = }\n{ age = 50 fnlwgt = 166220 education-num = 12 capital-gain = 3942 hours-per-week = 400_3 = 11_7 = 12_2 = 13_6 = 14_3 = 15_1 = 16_2 = 17_1 = }\n{ age = 47 fnlwgt = 167159 education-num = 9 capital-gain = 4650 hours-per-week = 400_3 = 11_2 = 12_3 = 13_10 = 14_1 = 15_1 = 16_1 = 17_1 = }\n{ age = 27 fnlwgt = 165365 education-num = 9 capital-gain = 2885 hours-per-week = 400_3 = 11_2 = 12_2 = 13_10 = 14_2 = 15_3 = 16_1 = 17_21 = }\n{ age = 33 fnlwgt = 164707 education-num = 12 capital-gain = 2174 hours-per-week = 550_3 = 11_7 = 12_1 = 13_2 = 14_5 = 15_1 = 16_2 = 17_5 =

In [11]:
explanation.cexemplars

'None'

In [12]:
explanation.feature_importance_all

{'age': 0.13549619930073203,
 ' fnlwgt': 0.0,
 ' education-num': 0.0802589102269836,
 ' capital-gain': 0.17233821563795834,
 ' capital-loss': 0.02310704115291639,
 ' hours-per-week': 0.01901018399317246,
 '0_1': 0.0014670974921870148,
 '0_2': 0.001766145639169358,
 '0_3': 0.014147750717697801,
 '0_4': 0.009975231576220392,
 '0_5': 0.0,
 '0_6': 0.0,
 '0_7': 0.007952362770577393,
 '0_8': 0.0,
 '0_9': 0.0,
 '1_1': 0.00951101462058233,
 '1_2': 0.013435975926484753,
 '1_3': 0.008931305266806638,
 '1_4': 0.0,
 '1_5': 0.0,
 '1_6': 0.009053610495885307,
 '1_7': 0.002827652872382836,
 '1_8': 0.0,
 '1_9': 0.02438204285629986,
 '1_10': 0.0,
 '1_11': 0.006656879735910171,
 '1_12': 0.0031176744604833133,
 '1_13': 0.009483677075180844,
 '1_14': 0.0027716936749532573,
 '1_15': 0.0030128912020547086,
 '1_16': 0.0,
 '2_1': 0.03449764093758946,
 '2_2': 0.08832971405149916,
 '2_3': 0.0,
 '2_4': 0.012813399458770245,
 '2_5': 0.003308660936684337,
 '2_6': 0.0029134957740201374,
 '2_7': 0.003171561481109120

In [13]:
explanation.feature_importance

{'age': 0.13549619930073203,
 ' education-num': 0.0802589102269836,
 ' capital-gain': 0.17233821563795834,
 '2_2': 0.08832971405149916,
 '3_2': 0.03421139679376052,
 '3_4': 0.013437046295648236,
 '4_4': 0.02575799867678644,
 'cluster': 9.801094978070428e-18}